In [2]:
# Neighborrhood Classification
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import io

In [3]:
html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [4]:
#turn the html into a beautiful soup text object

bsoup = BeautifulSoup(html.text, 'xml')

In [5]:
# extracting raw table (from webpage)
table = bsoup.find('table')
Postcode      = []
Borough       = []
Neighbourhood = []


In [6]:
#clean table extraction

for tr_cell in table.find_all('tr'):
  
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
        counter +=1

        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):
            continue
    
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
            pass

    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
            continue
    
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [7]:
#Finding the unique postcodes and joining the neighbourhoods together based on postcode

unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []
 
for unique_element in unique_p:
    p_var = '';
    b_var = ''; 
    n_var = ''; 
    for postcode_id, postcode_element in enumerate(Postcode):
        if (unique_element == postcode_element):
                p_var = postcode_element;
                b_var = Borough[postcode_id]
                if n_var == '': 
                    n_var = Neighbourhood[postcode_id]
                else:
                    n_var = n_var + ', ' + Neighbourhood[postcode_id]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


In [16]:
# dataframes todisplay 
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_tor = pd.DataFrame.from_dict(toronto_dict)
df_tor.to_csv('toronto_part1.csv')
df_tor.head(11)

,Postcode,Borough,Neighbourhood
0,M1X,Scarborough,Upper Rouge
1,M8Z,Etobicoke,"Mimico NW, The Queensway West"
2,M5C,Downtown Toronto,St. James Town
3,M2J,North York,Henry Farm
4,M4Y,Downtown Toronto,Church and Wellesley
5,M9M,North York,"Emery, Humberlea"
6,M1B,Scarborough,"Rouge, Malvern"
7,M4H,East York,Thorncliffe Park
8,M1K,Scarborough,"Ionview, Kennedy Park"
9,M6H,West Toronto,Dovercourt Village


In [9]:
df_tor.shape

(77, 3)

In [10]:
#getting the latitude and the longitude coordinates of each neighborhood.
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

# rename the first column to allow merging dataframes on Postcode
c.columns = ['Postcode', 'Latitude', 'Longitude']
df_tor = pd.merge(c, df_tor, on='Postcode')

# reorder column names and show the dataframe
df_tor = df_tor[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
df_tor.head(11)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
5,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029
6,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
7,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476
8,M1N,Scarborough,Birch Cliff,43.692657,-79.264848
9,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304


In [27]:
from sklearn.cluster import KMeans

df_tor1 = df_tor
df_tor2 = df_tor1[df_tor1['Borough'].str.contains('Toronto',regex=False)]
k=5
toronto_clustering = df_tor2.drop(['Postcode','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_

array([4, 4, 3, 3, 4, 4, 2, 2, 3, 3, 0, 3, 0, 0, 2, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1], dtype=int32)

In [ ]:
df_tor2.insert(0, 'Cluster Labels', kmeans.labels_)

In [29]:
df_tor2

,Cluster Labels,Postcode,Borough,Neighbourhood
2,3,M5C,Downtown Toronto,St. James Town
4,3,M4Y,Downtown Toronto,Church and Wellesley
9,1,M6H,West Toronto,Dovercourt Village
12,1,M4L,East Toronto,India Bazaar
13,3,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Sou..."
14,3,M4W,Downtown Toronto,Rosedale
23,4,M5J,Downtown Toronto,"Toronto Islands, Union Station"
25,4,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill"
28,1,M4K,East Toronto,Riverdale
30,1,M5R,Central Toronto,"The Annex, Yorkville"
